In [9]:
from langchain_community.llms import Ollama
from langchain_core.output_parsers import JsonOutputParser
from langchain.output_parsers import YamlOutputParser

In [10]:
llm = Ollama(model="gemma:7b")

In [11]:
from langchain_core.prompts import ChatPromptTemplate
import re

def extract_cypher_scripts(text):
    cypher_list = list()
    for line in text.split("\n"):
        if 'CREATE' in line or 'MATCH' in line or 'CONNECT' in line:
            cypher_list.append(line)
            
    return cypher_list

## CoT Knowledge Grapg Creation

In [12]:
import pandas as pd
import numpy as np
cypher_list = list()
contexts = pd.read_csv('/Users/mertakcay/Projects/KnowledgeGraphBuilder/context.csv')
context_outputs = list()
for context in contexts['context']:
    prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a ontologist. You build knowledge graph from plain text. \n 
     Task 1: Extract Entities from plain text as possible as atomic. \n
     Task 2: Define Cypher Node Type to each Entity.\n
     Task 3: Add features of each Entity from plain text. \n
     Task 4: Extract relationship between Entities from plain text as possible as atomic. \n\n
     Task 5: Convert all outputs to Neo4j Cypher Creation Format"""),
    ("system", "Let's think step by step."),
    ("user", "{input}")
    ])
    chain = prompt | llm 
    output = chain.invoke({"input": context})
    print(output)
    print('--------------------------------')
    cypher_scripts = extract_cypher_scripts(output.replace('`','"'))
    context_outputs.append(cypher_scripts)
    

## Knowledge Graph Building from Plain Text

**Task 1: Extracting Entities**

- **Entities:**
  - Beyoncé Giselle Knowles-Carter
  - Houston, Texas
  - Destiny's Child
  - R&B
  - Mathew Knowles

**Task 2: Defining Cypher Node Type:**

- **Beyoncé:** Actor, Singer, Songwriter, Record Producer
- **Houston:** City
- **Destiny's Child:** Group
- **R&B:** Genre
- **Mathew Knowles:** Manager

**Task 3: Adding Features:**

- **Beyoncé:**
  - Born in 1981
  - Raised in Houston
  - Performed in singing and dancing competitions
  - Released debut album "Dangerously in Love" in 2003
  - Won five Grammy Awards
  - Featured Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy"

**Task 4: Extracting Relationships:**

- **Beyoncé is born in Houston.**
- **Destiny's Child was formed in Houston.**
- **Mathew Knowles managed Destiny's Child.**
- **Destiny's Child became one of the world's best-selling girl groups.**
- **Beyoncé's debut album "Dangerously in Love" established her as a solo